In [1]:
!pip install psycopg2

In [2]:
import psycopg2
import pandas as pd

In [3]:
def create_database():
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thuan84826")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    cur.execute("DROP DATABASE IF EXISTS transactions")
    cur.execute("CREATE DATABASE transactions")
    
    conn.close()
        
    conn = psycopg2.connect("host=localhost dbname=transactions user=postgres password=thuan84826")
    cur = conn.cursor()
    return cur, conn


In [4]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [5]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [6]:
df = pd.read_csv('Retail_Transactions_Dataset.csv')

In [7]:
df.head(5)

,Transaction_ID,Date,Customer_Name,Product,Total_Items,Total_Cost,Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion
0,1000000000,12/21/2020 19:42,Cheyenne Newman,['Hair Gel'],6,12.77,Debit Card,New York,Convenience Store,True,Student,Winter,NaN
1,1000000001,7/6/2020 7:45,Emily Fitzgerald,"['Tuna', 'Bread', 'Tissues', 'Trash Bags']",5,13.88,Debit Card,Houston,Supermarket,False,Professional,Fall,BOGO (Buy One Get One)
2,1000000002,10/2/2021 6:28,Michael Webb,"['Jam', 'Soap', 'Ketchup']",7,47.02,Debit Card,Miami,Convenience Store,False,Young Adult,Winter,NaN
3,1000000003,1/10/2022 5:39,Kimberly Lin,['BBQ Sauce'],9,83.86,Mobile Payment,Seattle,Warehouse Club,True,Senior Citizen,Summer,Discount on Selected Items
4,1000000004,10/13/2021 7:28,Cathy Hernandez,"['Hand Sanitizer', 'Bread', 'Extension Cords',...",4,30.55,Debit Card,Houston,Warehouse Club,False,Senior Citizen,Spring,NaN


In [8]:
cur, conn = create_database() 

In [9]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.drop_duplicates().reset_index(drop=True)
df['trans_id'] = df.index
# df.head(5)

In [10]:
date_dim = df[['Date', 'Season']].reset_index(drop=True)
date_dim['date'] = date_dim['Date']#.reset_index(drop=True)
date_dim['season'] = date_dim['Season']
date_dim['date_id'] = date_dim.index

date_dim = date_dim[['date_id', 'date', 'season']]

# date_dim.head()

In [11]:
customer_dim = df[['Customer_Name','Customer_Category']].reset_index(drop=True)
customer_dim['Customer_Name'] = customer_dim['Customer_Name']
customer_dim['Customer_Category'] = customer_dim['Customer_Category']
customer_dim['Customer_Id'] = customer_dim.index
customer_dim = customer_dim[['Customer_Id','Customer_Name','Customer_Category']]

In [12]:
type_transaction_dim = df[['Discount_Applied', 'Promotion']].reset_index(drop=True)
type_transaction_dim['Discount_Applied'] = type_transaction_dim['Discount_Applied']
type_transaction_dim['Promotion'] = type_transaction_dim['Promotion']
type_transaction_dim['Type_Id'] = type_transaction_dim.index

type_transaction_dim = type_transaction_dim[['Type_Id', 'Discount_Applied','Promotion']]

In [13]:
payment_dim = df[['Payment_Method']].reset_index(drop=True)
payment_dim['Pay_Type_Id'] = payment_dim.index
payment_dim = payment_dim[['Pay_Type_Id','Payment_Method']]

In [14]:
location_dim = df[['City']].reset_index(drop=True)

location_dim['locate_id'] = location_dim.index
location_dim = location_dim[['locate_id','City']]

In [15]:
store_dim = df[['Store_Type']].reset_index(drop=True)

store_dim['store_id'] = store_dim.index
store_dim = store_dim[['store_id','Store_Type']]

In [58]:
date_dim_table_create = ("""CREATE TABLE IF NOT EXISTS date_dim_tb(
date_id int8 PRIMARY KEY,
date_time timestamp,
season VARCHAR

)
""")  

In [57]:
cur.execute("""DROP TABLE date_dim_tb
""")
conn.commit()

In [59]:
cur.execute(date_dim_table_create)
conn.commit()

In [40]:
customer_dim_table_create = ("""CREATE TABLE IF NOT EXISTS customer_dim_tb(
customer_id int8 PRIMARY KEY,
customer_name VARCHAR,
customer_category VARCHAR

)
""")  

In [41]:
cur.execute(customer_dim_table_create)
conn.commit()

In [18]:
type_transaction_dim_table_create = ("""CREATE TABLE IF NOT EXISTS type_transaction_dim_tb(
type_id int8 PRIMARY KEY,
discount bool,
promotion VARCHAR

)
""")  

In [19]:
cur.execute(type_transaction_dim_table_create)
conn.commit()

In [20]:
payment_dim_table_create = ("""CREATE TABLE IF NOT EXISTS payment_dim_tb(
pay_type_id int8 PRIMARY KEY,
payment_method VARCHAR

)
""")  

In [21]:
cur.execute(payment_dim_table_create)
conn.commit()

In [22]:
location_dim_table_create = ("""CREATE TABLE IF NOT EXISTS location_dim_tb(
location_id int8 PRIMARY KEY,
city VARCHAR
)
""")  

In [23]:
cur.execute(location_dim_table_create)
conn.commit()

In [24]:
store_dim_table_create = ("""CREATE TABLE IF NOT EXISTS store_dim_tb(
store_id int8 PRIMARY KEY,
store_type_name VARCHAR

)
""")  

In [25]:
cur.execute(store_dim_table_create)
conn.commit()

In [26]:
date_dim_table_insert = ("""INSERT INTO date_dim_tb(
date_id,
date_time,
season)
VALUES (%s,%s,%s)
""")

# for i, row in date_dim.iterrows():
#     cur.execute(date_dim_table_insert, list(row))

In [60]:
for i, row in date_dim.iterrows():
#     print(list(row))
    cur.execute(date_dim_table_insert, list(row))

In [62]:
customer_dim_table_insert = ("""INSERT INTO customer_dim_tb(
customer_id,
customer_name,
customer_category)
VALUES (%s,%s,%s)
""")

for i, row in customer_dim.iterrows():
    cur.execute(customer_dim_table_insert, list(row))

In [64]:
type_transaction_dim_table_insert = ("""INSERT INTO type_transaction_dim_tb(
type_id,
discount,
promotion)
VALUES (%s,%s,%s)
""")

for i, row in type_transaction_dim.iterrows():
    cur.execute(type_transaction_dim_table_insert, list(row))

In [66]:
payment_dim_table_insert = ("""INSERT INTO payment_dim_tb(
pay_type_id,
payment_method)
VALUES (%s,%s)
""")

for i, row in payment_dim.iterrows():
    cur.execute(payment_dim_table_insert, list(row))

In [71]:
location_dim_table_insert = ("""INSERT INTO location_dim_tb(
location_id,
city)
VALUES (%s,%s)
""")

for i, row in location_dim.iterrows():
    cur.execute(location_dim_table_insert, list(row))

In [73]:
store_dim_table_insert = ("""INSERT INTO store_dim_tb(
store_id,
store_type_name)
VALUES (%s,%s)
""")

for i, row in store_dim.iterrows():
    cur.execute(store_dim_table_insert, list(row))

In [74]:
conn.commit()

In [70]:
conn.rollback()